In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

/Users/TimScholes/anaconda/lib/python3.5/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [2]:
# Some global variables for smoothing
x_bottom_left_main_prev = 0
x_middle_left_main_prev = 0
x_bottom_right_main_prev = 0
x_middle_right_main_prev = 0
counter = 0

In [5]:
def process_image(image):

    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    # Define a kernel size and apply Gaussian smoothing
    ######kernel_size = 5
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    ##vertices = np.array([[(0,imshape[0]),(0, 0), (imshape[1], 0), (imshape[1],imshape[0])]], dtype=np.int32)
    ##########vertices = np.array([[(0,imshape[0]),(450, 290), (490, 290), (imshape[1],imshape[0])]], dtype=np.int32)
    vertices = np.array([[(100,imshape[0]),(470, 305), (470, 305), (970,imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    ######min_line_length = 40 #minimum number of pixels making up a line
    min_line_length = 70 #minimum number of pixels making up a line
    #######max_line_gap = 20    # maximum gap in pixels between connectable line segments
    max_line_gap = 30    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

####################################################################################
# Iterate over the output "lines" and draw lines on a blank image
#     for line in lines:
#         for x1,y1,x2,y2 in line:
#            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)

    # Variables
    
    left_side_counter = 0
    right_side_counter = 0

    x_bottom_left_total = 0
    x_middle_left_total = 0
    x_bottom_left_main = 0
    x_middle_left_main = 0

    x_bottom_right_total = 0
    x_middle_right_total = 0
    x_bottom_right_main = 0
    x_middle_right_main = 0
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            gradient = ((y2-y1)/(x2-x1))
            if gradient == 0:
                continue
                 
            ## new line: y-mx=b
            intercept = y1 - gradient*x1

            # find x when y at full extent - point 1 = (x, imshape[0])
            x_bottom = int((imshape[0] - intercept)/gradient)

            # find x when y at perspective limit - point 1 = (x, 390)
            x_middle = int((350 - intercept)/gradient)
            
            ##if gradient > 0:
            if gradient > 0.5 and gradient < 0.85:    
                x_bottom_left_total += x_bottom
                x_middle_left_total += x_middle
                left_side_counter += 1 
            elif gradient < -0.5 and gradient > -0.85:
                x_bottom_right_total += x_bottom
                x_middle_right_total += x_middle
                right_side_counter += 1

    global counter
    global x_bottom_left_main_prev
    global x_middle_left_main_prev
    global x_bottom_right_main_prev
    global x_middle_right_main_prev
                       
    #print('global = {}'.format(counter))
    
    # left line
    if left_side_counter > 0:
        if counter > 0:
            x_bottom_left_main = (int(x_bottom_left_total/left_side_counter)+ x_bottom_left_main_prev)//2
            x_middle_left_main = (int(x_middle_left_total/left_side_counter)+ x_middle_left_main_prev)//2
        else:
            x_bottom_left_main = int(x_bottom_left_total/left_side_counter)
            x_middle_left_main = int(x_middle_left_total/left_side_counter)
            
        cv2.line(line_image,(x_bottom_left_main, imshape[0]),(x_middle_left_main, 350),(255,50,0),30)
        x_bottom_left_main_prev = x_bottom_left_main
        x_middle_left_main_prev = x_middle_left_main
        counter += 1 

    # right line
    if right_side_counter > 0:
        if counter > 0:
            x_bottom_right_main = int(x_bottom_right_total/right_side_counter)
            x_middle_right_main = int(x_middle_right_total/right_side_counter)
        else:
            x_bottom_right_main = int(x_bottom_right_total/right_side_counter)
            x_middle_right_main = int(x_middle_right_total/right_side_counter)
        
        cv2.line(line_image,(x_bottom_right_main, imshape[0]),(x_middle_right_main, 350),(255,50,0),30)  
        x_bottom_right_main_prev = x_bottom_left_main
        x_middle_right_main_prev = x_middle_left_main
        counter += 1 
                
####################################################################################   

    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges)) 

    # Draw the lines on the edge image
    new_image = cv2.addWeighted(image, 0.8, line_image, 1, 0) 

    return new_image

In [7]:
final_output = 'solidWhiteRight_output_1.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
final_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time final_clip.write_videofile(final_output, audio=False)


MoviePy: building video file solidWhiteRight_output_1.mp4
----------------------------------------

Writing video into solidWhiteRight_output_1.mp4
|----------| 0/213   0% [elapsed: 00:00 left: ?, ? iters/sec]

/Users/TimScholes/anaconda/lib/python3.5/site-packages/moviepy/audio/io/readers.py:110: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.nchannels))


Done writing video in solidWhiteRight_output_1.mp4 !
Your video is ready !
CPU times: user 3.06 s, sys: 804 ms, total: 3.86 s
Wall time: 6.13 s
